In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import json

import pymysql
import sqlalchemy
from urllib import parse
from dotenv import load_dotenv

import os 
import sqlalchemy

# load .env
load_dotenv(".env")

db_name = os.environ.get('DB_NAME')
db_user = os.environ.get('DB_USER')
db_password = os.environ.get('DB_PASSWORD')
db_host = os.environ.get('DB_HOST')
db_port = os.environ.get('DB_PORT')

engine = sqlalchemy.create_engine(f"mysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

In [ ]:
# 데이터 불러오기
df = pd.read_sql_table("final_items", engine)

In [ ]:
head = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"}

In [ ]:
wine_id = df['wine_id']

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# wine_id 리스트에서 상위 10개만 추출 (테스트용)
wine_id = df['wine_id'].tolist()[:10]  # df['wine_id']에서 상위 10개 추출

# 이미지를 크롤링하는 함수
def fetch_image_url(wine_id):
    try:
        # Vivino API나 웹 페이지 URL
        # url = f"http://www.vivino.com/w/{wine_id}"
        url = f"http://www.vivino.com/api/w/{wine_id}"

        
        # HTTP 요청 보내기
        head = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"}
        r= requests.get(url, headers=head)
        soup = BeautifulSoup(r.text, 'html.parser')
        
        # 'thumbs/'가 포함된 이미지 URL을 추출하는 정규표현식
        pattern = r'src="(//[^"]*thumbs/[^"]+)"'
        match = re.search(pattern, str(soup))

        # 매칭된 이미지 URL을 반환, 없으면 None
        if match:
            img_url = match.group(1)
            return img_url
        else:
            return None
    except Exception as e:
        print(f"Error fetching wine_id {wine_id}: {e}")
        return None

# 빈 리스트를 만들어 img_url 데이터를 저장할 준비
img_urls = []

# ThreadPoolExecutor를 사용하여 병렬 처리 (테스트로 10개만 실행)
with ThreadPoolExecutor(max_workers=5) as executor:  # 테스트에서는 5개 워커로 실행
    # tqdm을 사용하여 진행 상황 표시
    futures = [executor.submit(fetch_image_url, wid) for wid in wine_id]
    
    # 결과 처리 (tqdm으로 진행상황 표시)
    for future in tqdm(as_completed(futures), total=len(futures)):
        img_urls.append(future.result())

# df에서 상위 10개의 데이터프레임만 선택하여 img_url 추가
df_test = df.head(10).copy()  # 상위 10개의 행을 가져와서 테스트
df_test['img_url'] = img_urls  # img_url 추가

# 결과 확인
print(df_test['img_url'])


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# 전체 wine_id 리스트
wine_id = df['wine_id'].tolist()  # df['wine_id']에서 전체 wine_id 추출

# 이미지 URL과 브랜드 이름을 크롤링하는 함수
def fetch_wine_data(wine_id):
    try:
        # Vivino 웹 페이지 URL
        url = f"http://www.vivino.com/w/{wine_id}"
        
        # HTTP 요청 보내기
        headers = {
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
        }
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')

        # 1. 600x600 이미지 URL 추출
        image_pattern = r'//[^"]*thumbs/[^"]+'
        image_match = re.search(image_pattern, str(soup))
        img_url = 'https:' + image_match.group(0) if image_match else None
        
        # 2. 브랜드 이름 추출
        brand_pattern = r'"winery":\{"id":[^,]+,"name":"([^"]+)"'
        brand_match = re.search(brand_pattern, str(soup))
        brand_name = brand_match.group(1) if brand_match else None

        return img_url, brand_name
    
    except Exception as e:
        print(f"Error fetching wine_id {wine_id}: {e}")
        return None, None

# 빈 리스트를 만들어 데이터를 저장할 준비
img_urls = []
brand_names = []

# ThreadPoolExecutor를 사용하여 병렬 처리
with ThreadPoolExecutor(max_workers=10) as executor:  # 전체 데이터에 대해 10개의 워커를 사용하여 실행
    # tqdm을 사용하여 진행 상황 표시
    futures = [executor.submit(fetch_wine_data, wid) for wid in wine_id]
    
    # 결과 처리 (tqdm으로 진행상황 표시)
    for future in tqdm(as_completed(futures), total=len(futures)):
        img_url, brand_name = future.result()
        img_urls.append(img_url)
        brand_names.append(brand_name)

# df에서 전체 wine_id에 대한 데이터프레임에 img_url 및 brand_name 추가
df['img_url'] = img_urls  # img_url 추가
df['brand_name'] = brand_names  # brand_name 추가

# 결과 확인
# print(df[['wine_id', 'img_url', 'brand_name']])

In [ ]:
# 데이터프레임을 MySQL에 저장
df.to_sql('items_with_img', con=engine, if_exists='replace', index=False)